In [18]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

url_target = 'https://standings.stalruth.dev/2025/regional-stuttgart/masters'
page = urlopen(url_target)
soup = BeautifulSoup(page)
day1_ppl = 690
day2_ppl = 96
event_prefix = 'stuttgart25'

In [19]:
standing_table = soup.findAll('table')[1]

In [20]:
standing_rows = standing_table.findAll('tr')

In [21]:
standing_rows[0].find('td', attrs={'class': 'name-cell'})

<td class="name-cell">
<a href="/2025/regional-stuttgart/masters/576">
              Eden Batchelor [UK]
            </a>
</td>

In [22]:
mons_usage_wr_dict = {}


In [23]:
drop_form_names = ['Sinistcha', 'Tatsugiri', 'Maushold']

In [24]:
for r in standing_rows:
    mons_ = r.find_all('span', attrs={'class': 'pokemon-icon'})
    record_ = r.find('td', attrs={'class': 'record-resistance'}).find('p', attrs={'class': 'record'}).get_text()
    for m in mons_:
        mon_name = m.get('title')
        # Drop the form names
        for dpn in drop_form_names:
            if dpn in mon_name:
                mon_name = dpn
        wins = int(record_.split('-')[0])
        losses = int(record_.split('-')[1].split('*')[0])
        if not (mon_name in mons_usage_wr_dict.keys()):
            mons_usage_wr_dict[mon_name] = {'win': wins,
                                    'loss': losses,
                                    'day1': 1,
                                    'day2': 1 if (wins > 5) else 0
                                    }
        else:
            # print(mons_usage_wr_dict[mon_name], record_)
            mons_usage_wr_dict_temp = {'win': mons_usage_wr_dict[mon_name]['win'] + wins,
                                        'loss': mons_usage_wr_dict[mon_name]['loss'] + losses,
                                        'day1': mons_usage_wr_dict[mon_name]['day1'] + 1,
                                        'day2': mons_usage_wr_dict[mon_name]['day2'] + (1 if (wins > 5) else 0)
                                        }
            mons_usage_wr_dict[mon_name] = mons_usage_wr_dict_temp


In [25]:
mons_usage_wr_dict

{'Sneasler': {'win': 1089, 'loss': 1124, 'day1': 276, 'day2': 37},
 'Ursaluna-Bloodmoon': {'win': 680, 'loss': 605, 'day1': 156, 'day2': 24},
 'Dragonite': {'win': 865, 'loss': 744, 'day1': 191, 'day2': 38},
 'Gholdengo': {'win': 970, 'loss': 864, 'day1': 218, 'day2': 42},
 'Amoonguss': {'win': 484, 'loss': 445, 'day1': 110, 'day2': 26},
 'Incineroar': {'win': 1141, 'loss': 1076, 'day1': 272, 'day2': 45},
 'Archaludon': {'win': 706, 'loss': 651, 'day1': 163, 'day2': 33},
 'Pelipper': {'win': 622, 'loss': 574, 'day1': 144, 'day2': 31},
 'Basculegion': {'win': 440, 'loss': 392, 'day1': 101, 'day2': 20},
 'Ursaluna': {'win': 532, 'loss': 476, 'day1': 122, 'day2': 23},
 'Hydreigon': {'win': 70, 'loss': 91, 'day1': 20, 'day2': 2},
 'Porygon2': {'win': 270, 'loss': 287, 'day1': 71, 'day2': 8},
 'Kingambit': {'win': 440, 'loss': 470, 'day1': 116, 'day2': 10},
 'Volcarona': {'win': 345, 'loss': 309, 'day1': 78, 'day2': 13},
 'Rillaboom': {'win': 859, 'loss': 874, 'day1': 213, 'day2': 26},
 'El

In [26]:
mons_usage_winrate_df = pd.DataFrame(mons_usage_wr_dict).T

In [27]:
mons_usage_winrate_df

,win,loss,day1,day2
Sneasler,1089,1124,276,37
Ursaluna-Bloodmoon,680,605,156,24
Dragonite,865,744,191,38
Gholdengo,970,864,218,42
Amoonguss,484,445,110,26
...,...,...,...,...
Poliwrath,1,7,1,0
Conkeldurr,0,7,1,0
Morpeko,0,4,1,0
Lucario,0,5,1,0


In [28]:
mons_usage_winrate_df['win_rate'] = mons_usage_winrate_df['win']/(mons_usage_winrate_df['win'] + mons_usage_winrate_df['loss'])

In [29]:
mons_usage_winrate_df['conversion_rate(day2:day1)'] = (mons_usage_winrate_df['day2']/day2_ppl) / (mons_usage_winrate_df['day1']/day1_ppl)

In [30]:
mons_usage_winrate_df['day1']

Sneasler              276
Ursaluna-Bloodmoon    156
Dragonite             191
Gholdengo             218
Amoonguss             110
                     ... 
Poliwrath               1
Conkeldurr              1
Morpeko                 1
Lucario                 1
Lapras                  1
Name: day1, Length: 149, dtype: int64

In [31]:
mons_usage_winrate_df = mons_usage_winrate_df.sort_values(by=['day2', 'conversion_rate(day2:day1)'], ascending=False)

In [32]:
mons_usage_winrate_df = mons_usage_winrate_df.round(4)

In [33]:
mons_usage_winrate_df

,win,loss,day1,day2,win_rate,conversion_rate(day2:day1)
Incineroar,1141,1076,272,45,0.5147,1.1891
Gholdengo,970,864,218,42,0.5289,1.3847
Dragonite,865,744,191,38,0.5376,1.4300
Sneasler,1089,1124,276,37,0.4921,0.9635
Archaludon,706,651,163,33,0.5203,1.4551
...,...,...,...,...,...,...
Poliwrath,1,7,1,0,0.1250,0.0000
Conkeldurr,0,7,1,0,0.0000,0.0000
Morpeko,0,4,1,0,0.0000,0.0000
Lucario,0,5,1,0,0.0000,0.0000


In [34]:
mons_usage_winrate_df.to_csv('output_txt/{}_mons_summary.csv'.format(event_prefix))